In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymongo
!pip install nltk
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import re
import time
# file management
import os
import glob
import time
#file saving
import json
#utilities
from nltk.util import ngrams
from pymongo import MongoClient
from collections import Counter
from tqdm import tqdm
from itertools import islice


import psutil
from multiprocessing import Pool
import multiprocessing
import itertools

In [4]:
WORKING_DIR = '/content/drive/MyDrive/generate_n_grams'

In [5]:
file_name = WORKING_DIR + "/wikimatrix.en"

In [6]:
!pwd

/content


In [7]:
def generate_directories():
  dir_name= WORKING_DIR + '/ngrams_multiprocessing_nosave_benchmark/'
  for i in range(2,7):
    os.makedirs(dir_name + str(i))



In [8]:
generate_directories()

In [9]:
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r   

In [10]:
dir_name=WORKING_DIR + '/ngrams_multiprocessing_nosave_benchmark/'
dir_list = os.listdir(dir_name)
print(dir_list)
# f = []
# for (dirpath, dirnames, filenames) in os.walk(dir_name):
#     print(filenames)


['2', '3', '4', '5', '6']


In [11]:
def append_dict(list_of_counts):
  counter_total = Counter({})
  for dict_count in list_of_counts:
     counter = Counter(dict_count)
     counter_total += counter
  return counter_total

In [ ]:
def process_files_dict(dir_name):
  dir_list = os.listdir(dir_name)
  for n_gram_dir in dir_list:
    change_dir = dir_name + n_gram_dir
    #os.chdir(change_dir)
    files = [file for file in glob.glob(change_dir + '/' + "*.json")]
    if len(files) > 1:
      main_file_dir = change_dir + '/total_n_gram.json'
      for file_name in files:
        if file_name != main_file_dir:
          main_json = open(main_file_dir, "r")
          other_json = open(file_name, "r")
          main_dict = json.loads(main_json.read())
          other_dict = json.loads(other_json.read())
          list_dicts = [main_dict, other_dict]
          key_count = append_dict(list_dicts)
          key_count = dict(key_count)
          main_json.close()
          other_json.close()
          with open(main_file_dir, 'w') as fp:
            json.dump(key_count, fp)
          if os.path.exists(file_name):
            os.remove(file_name)
          else:
            print("{} does not exist".format(file_name))
    else:
      pass

    

    



In [ ]:
process_files_dict(dir_name)

FileNotFoundError: ignored

In [ ]:
def get_dict(counter_item):
  text= ' '.join(counter_item[0])
  count = counter_item[1]
  count_dict = {text: count}
  return count_dict

In [ ]:
def get_counts(counter):
  reference_dict = {}
  for item in counter.most_common():
    text= ' '.join(item[0])
    count = item[1]
    if not reference_dict.get(text,""):
      reference_dict[text] = count
  return reference_dict

In [ ]:
def append_dict(list_of_counts):
  counter_total = Counter({})
  for dict_count in list_of_counts:
     counter = Counter(dict_count)
     counter_total += counter
  return counter_total

In [ ]:
def count_in_text(text ,store):
  for i in range(2,7):
    ngram_counts = Counter(ngrams(text.split(), i))
    temp_dict = get_counts(ngram_counts)
    store.get(i).append(temp_dict)


In [ ]:
def chunks(data, SIZE=100000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

In [ ]:
def count_in_text(text ,store):
  for i in range(2,7):
    ngram_counts = Counter(ngrams(text.split(), i))
    temp_dict = get_counts(ngram_counts)
    store.get(i).append(temp_dict)

In [ ]:
def count_in_text_local(batch_num, text, dir_name_template):
  for i in range(2,7):
    ngram_counts = Counter(ngrams(text.split(), i))
    temp_dict = get_counts(ngram_counts)
    dir_name = dir_name_template + str(i)
    with open(dir_name + '/{}.json'.format(batch_num), 'w') as fp:
      json.dump(temp_dict, fp)
    print('saved batch {batch_num} for {i} grams'.format(batch_num=batch_num, i=i))


In [ ]:
# def count_in_text_local(batch_num, text, dir_name_template):
#   all_vars = [batch_num, text, dir_name_template]
#   #pool = Pool(psutil.cpu_count(logical=False))
#   def count_per_ngram(i):
#     ngram_counts = Counter(ngrams(text.split(), i))
#     temp_dict = get_counts(ngram_counts)
#     dir_name = dir_name_template + str(i)
#     with open(dir_name + '/{}.json'.format(batch_num), 'w') as fp:
#       json.dump(temp_dict, fp)
#     print('saved batch {batch_num} for {i} grams'.format(batch_num=batch_num, i=i))
#   pool = multiprocessing.Pool(5)
#   pool.map(count_per_ngram, range(2,7))
#   pool.close()  
  


In [ ]:
def count_in_text_local(batch_num, text, dir_name_template, i):
  ngram_counts = Counter(ngrams(text.split(), i))
  temp_dict = get_counts(ngram_counts)
  dir_name = dir_name_template + str(i)
  with open(dir_name + '/{}.json'.format(batch_num), 'w') as fp:
    json.dump(temp_dict, fp)
  print('saved batch {batch_num} for {i} grams'.format(batch_num=batch_num, i=i))


In [ ]:
# def aggregate_counts(save_dir):
#   for i in range(2,7):
#     #os.makedirs(dir_name + str(i))


In [ ]:
def get_ngram_dicts(filename, save_dir):
  #ngram_dicts = {2: [], 3:[], 4:[], 5:[], 6:[]}
  with open(filename) as fp:
      text = ''
      count = 0
      segment_number = 1
      grams_n = [2,3,4,5,6]
      while True:
        line = fp.readline()
        if not line:
          if segment_number == 1:
            file_name = 'total_n_gram'
            params = [(file_name, text, save_dir, num) for num in grams_n]
            with Pool(processes=8) as pool:
              pool.starmap(count_in_text_local, params)
            #count_in_text_local('total_n_gram', text ,save_dir)
          else:
            file_name = segment_number
            params = [(file_name, text, save_dir, num) for num in grams_n]
            with Pool(processes=8) as pool:
              pool.starmap(count_in_text_local, params)
            #count_in_text_local(segment_number, text ,save_dir)
          #process_files_dict(save_dir)
          break
        text += line
        count += 1
        if count == 100000:
          if segment_number == 1:
            file_name = 'total_n_gram'
            params = [(file_name, text, save_dir, num) for num in grams_n]
            with Pool(processes=8) as pool:
              pool.starmap(count_in_text_local, params)
            #count_in_text_local('total_n_gram', text ,save_dir)
          else:
            file_name = segment_number
            params = [(file_name, text, save_dir, num) for num in grams_n]
            with Pool(processes=8) as pool:
              pool.starmap(count_in_text_local, params)
            #count_in_text_local(segment_number, text ,save_dir)
          #process_files_dict(save_dir)
          count = 0
          segment_number += 1 
          text = ''

      

          

            

In [ ]:
def upload_ngram_dicts(ngram_dicts, mongodb_collections):
  for index, key in enumerate(ngram_dicts):
    key_count = append_dict(ngram_dicts[key])
    key_count = dict(key_count)
    count = 0
    for item in chunks(key_count, 200000):
      staging_item = {'part': count, 'counter': item}
      mongodb_collections[index].insert_one(staging_item)
      count+=1
  


In [ ]:
n_cores = multiprocessing.cpu_count()
n_cores

8

In [ ]:
def main():
  dir_name = WORKING_DIR + '/ngrams_multiprocessing_nosave_benchmark/'
  get_ngram_dicts(file_name, dir_name)
  #upload_ngram_dicts(ngram_dicts, collections)


In [ ]:
start = time.time()
print("process starting")
main()
end = time.time()
print(end - start)


process starting
saved batch total_n_gram for 2 grams
saved batch total_n_gram for 3 grams
saved batch total_n_gram for 4 grams
saved batch total_n_gram for 5 grams
saved batch total_n_gram for 6 grams
saved batch 2 for 2 grams
saved batch 2 for 3 grams
saved batch 2 for 4 grams
saved batch 2 for 5 grams
saved batch 2 for 6 grams
saved batch 3 for 2 grams
saved batch 3 for 3 grams
saved batch 3 for 4 grams
saved batch 3 for 5 grams
saved batch 3 for 6 grams
saved batch 4 for 2 grams
saved batch 4 for 3 grams
saved batch 4 for 4 grams
saved batch 4 for 5 grams
saved batch 4 for 6 grams
29.716152667999268


In [ ]:
#26s